In [89]:
import pandas as pd
import numpy as np
import os

from folium.features import DivIcon
import folium
import pyproj
from pyproj import Proj, transform

%matplotlib inline  
import matplotlib.pyplot as plt 
import seaborn as sns 

import warnings
warnings.filterwarnings(action='ignore')

def katec_to_wgs84(x, y):
    inProj  = Proj('+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
    outProj = Proj({ 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84' })
    return transform( inProj, outProj, x, y )

WGS84 = { 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84', }
KATEC = { 'proj':'tmerc', 'lat_0':'38N', 'lon_0':'128E', 'ellps':'bessel','x_0':'400000', 'y_0':'600000', 'k':'0.9999','towgs84':'-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43'}
def wgs84_to_katec(longitude, latitude):
    return transform( Proj(**WGS84), Proj(**KATEC), longitude, latitude )

##### 현재 작업 위치 확인


In [2]:
pwd

'C:\\Users\\dudtj\\busan_moving_person\\custom_code'

In [97]:
cd ../data/csv_data/50/

C:\Users\dudtj\busan_moving_person\data\csv_data\50


In [1]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: CE27-0A82

 C:\Users\dudtj\busan_moving_person\custom_code 디렉터리

2021-08-16  오후 10:03    <DIR>          .
2021-08-16  오후 10:03    <DIR>          ..
2021-08-16  오후 09:57    <DIR>          .ipynb_checkpoints
2021-08-11  오후 06:41            49,516 0. file_column_dtype_val.ipynb
2021-08-12  오전 11:06            16,497 1. CVcell_admdong_cd.ipynb
2021-08-16  오후 05:05            17,076 10. rawdata_to_custom_cell_code.ipynb
2021-08-11  오후 05:06             4,926 11. add_admi_cd.ipynb
2021-08-12  오전 11:08            12,472 2. CVcell_admdong_cd_Match_admdong_nm.ipynb
2021-08-12  오후 04:50            37,512 3. dataset_fp.ipynb
2021-08-13  오후 05:53            52,252 4. dataset_50fp.ipynb
2021-08-12  오후 04:50            11,410 5. fill_na_code.ipynb
2021-08-13  오후 05:59             7,940 6. 50cell_fpop_dense.ipynb
2021-08-13  오후 06:21             8,012 7. 100cell_fpop_dense.ipynb
2021-08-13  오후 06:21             8,078 8. 300cell_fpop_dense.ipynb
2021-08-12  오후 08:41 

making success :)
function success :)
done :)
Wall time: 1min 27s


In [ ]:
def f_color(x):
    if x > geodf[geodf['total'] > geodf['total'].describe()[6]]['total'].describe()[6]:        # 기존 75%를 넘는 범위에서의 4사분위수 
        a = '#FF4F00'        
    elif x > geodf['total'].describe()[6]:      # 75%를 기준으로 설정
        a = '#FCB100'   
    elif x > geodf['total'].describe()[5]:      # 50%를 기준으로 설정
        a = '#E0F500' 
    elif x > geodf['total'].describe()[4]:      # 25%를 기준으로 설정
        a = '#8CF700'  
    else:
        a = '#00F000'
    return a

geodf['cell_color']= geodf['total'].apply(lambda x : f_color(x))
print("function success :)")
###################################################################

cell_size = 50
a = cell_size//2
x = geodf
x['nwx'], x['nwy'] = x['x']-a, x['y']+a
x['nex'], x['ney'] = x['x']+a, x['y']+a
x['swx'], x['swy'] = x['x']-a, x['y']-a
x['sex'], x['sey'] = x['x']+a ,x['y']-a

x['lng'], x['lat'] = katec_to_wgs84(x.x.to_list(), x.y.to_list())
x['nwlng'], x['nwlat'] = katec_to_wgs84(x.nwx.to_list(), x.nwy.to_list())
x['nelng'], x['nelat'] = katec_to_wgs84(x.nex.to_list(), x.ney.to_list())
x['swlng'], x['swlat'] = katec_to_wgs84(x.swx.to_list(), x.swy.to_list())
x['selng'], x['selat'] = katec_to_wgs84(x.sex.to_list(), x.sey.to_list())

map = folium.Map(location=[geodf['lat'].mean(), geodf['lng'].mean()], zoom_start=13)#, tiles='stamentoner')

for n in x.index:
    nw, ne, sw, se = [x.loc[n, 'nwlat'],x.loc[n, 'nwlng']],[x.loc[n, 'nelat'],x.loc[n, 'nelng']],[x.loc[n, 'swlat'],x.loc[n, 'swlng']],[x.loc[n, 'selat'],x.loc[n, 'selng']]
    cell_color = x.loc[n, 'cell_color']
    loc_name = 'ID:'+str(round(x.loc[n, 'id'],3)) + '       Total:'+ str(round(x.loc[n, 'total'],3))
    folium.Polygon(locations=[nw, sw, se, ne],
                   color='white',
                   popup=loc_name,
                   weight=0.55,
                   fill=True,
                   fill_color=cell_color,
                   fill_opacity=0.6
     ).add_to(map)

map.save('./50dense_html/50c_201912_busan_dense.html') ###### 이름변경필요부분
# map
print("done :)")